# Creating a Cutout with the MREL (ECHOWAVE) dataset

This walkthrough describes the process of creating a cutout using the [MREL (ECHOWAVE)](https://opendap.4tu.nl/thredds/catalog/data2/djht/f359cd0f-d135-416c-9118-e79dccba57b9/1/catalog.html). 

The repository contains datasets for multiple years, ranging from 1990 to 2021 for the region of North-West Europe. A downloaded ECHOWAVE dataset contains extensive information on wave, wind and currents variables and more. For the purposes of the `mrel_wave` module only two wave components are used: `hs: wave height (m)`, and `fp: peak wave frequency (Hz)` (which is transformed into `tp: wave peak period (m)`). These provide the combinations of climate variables used in the `Power_Matrix` to convert metocean data to power.

> **Notes**:
>
> 1. For creating a cutout from this dataset, you need to download large files and your computers memory needs to be able to handle these as well.
>
> 2. The `era5` module has been updated aswell to include the same wave variables under the `wave` feature, so a user has the option to use wave energy converters with an ERA5 dataset. Their memory usage is significantly lower as the grid resolution is also lower.

## Downloading the data set

To download the dataset from the TUDelft repository (OPeNDAP data service):

https://opendap.4tu.nl/thredds/catalog/data2/djht/f359cd0f-d135-416c-9118-e79dccba57b9/1/catalog.html

Variables that are downloaded 

| variable | time span
| --- | --- |
| MAPSTA | status map
| crs | None
| dpt | depth
| ucur | eastward current
| vcur | northward current
| uwnd | eastward_wind
| vwnd | northward_wind
| wlv | sea surface height above sea level
| ice | sea ice area fraction
| <mark style="background-color: lightblue">hs</mark> | <mark style="background-color: lightblue">significant height of wind and swell waves</mark>
| lm | mean wave length
| t02 | mean period T02
| t01 | mean period T01
| <mark style="background-color: lightblue">fp</mark> | <mark style="background-color: lightblue">wave peak frequency</mark>
| dir | wave mean direction
| spr | directional spread
| dp | peak direction
| phs0 | wave significant height partition 0
| phs1 | wave significant height partition 1
| phs2 | wave significant height partition 2
| ptp0 | peak period partition 0
| ptp1 | peak period partition 1
| ptp2 | peak period partition 2
| pdir0 | wave mean direction partition 0
| pdir1 | wave mean direction partition 1
| pdir2 | wave mean direction partition 2
| pdp0 | peak direction partition 0
| pdp1 | peak direction partition 1
| pdp2 | peak direction partition 2

* Directly download each file manually and store them in a new folder.
* Use the script below to merge the datasets into one, and keep only the used variables (highlighted above).

The **documentation** of the datasets can be found [here](https://data.4tu.nl/datasets/f359cd0f-d135-416c-9118-e79dccba57b9/1)

In [ ]:
import os 
import xarray as xr

# edit your folder path
your_directory = "path/to/folder"

files = [os.path.join(your_directory, f) for f in os.listdir(your_directory) if f.endswith(".nc")]
datasets = [xr.open_dataset(f, chunks={"time": 100})[["fp", "hs"]] for f in files]

ds = xr.concat(datasets, dim="time")

# example years
year = '2018'
month = '01-03'
ds.to_netcdf(os.path.join(your_directory, f"TU-MREL-2M_{year}_{month}.nc"))

## Specifying the cutout

Import the package and set recommended logging settings:

In [1]:
import logging

import atlite

logging.basicConfig(level=logging.INFO)

In [2]:
cutout = atlite.Cutout(
    path="...path/to/mrel_NW_europe_2018_02.nc",
    data_path="...path/to/TU-MREL_EU_ATL-2M_201802.nc",
    module="mrel_wave",
    x=slice(-12., 9.),
    y=slice(40., 62.),
    dx = 0.03,
    dy = 0.03,
    time=slice("2018-02-01", "2018-02-28"),
)

INFO:atlite.cutout:Building new cutout ...path\to\mrel_NW_europe_2018_02.nc


## Comparison with ERA5

The ECHOWAVE dataset has a horizontal resolution of 0.03 degrees per direction. This results in a better representation of the coastlines, which is critical for wave devices deployment. 

Comparing it with the ERA5 dataset, which has a resolution of 0.3 degrees, we see that the gaps of ECHOWAVE near the shores are smaller, but also the level of accuracy is higher. In the picture below, which shows Northern Ireland, we can see that the incoming waves from the Atlantic have significantly higher wave height `hs` values in the ECHOWAVE dataset, while those resources can reach the coasts of the island. On the other hand ERA5, has a difficulty showcasing both coastline and resource accuracy. This is a significant improvement for wave device deployment as it increases their availability.


![Echowave](mrel_era5.png)

Let's see what the available features that is the available weather data variables are.

In [ ]:
cutout.available_features.to_frame()

## Preparing the Cutout

No matter which dataset you use, this is where all the work actually happens.
This can be fast or take some or a lot of time and resources, among others depending on
your computer ressources.

In [ ]:
cutout.prepare()

Querying the cutout gives us basic information on which data is contained and can already be used.

## Inspecting the Cutout

In [ ]:
cutout  # basic information

In [ ]:
cutout.data.attrs  # cutout meta data

In [ ]:
cutout.prepared_features  # included weather variables

In [ ]:
cutout.data  # access to underlying xarray data